# Langroid Implementation

langroid:
tokenizer: https://github.com/simonw/ttok/issues/8

In [ ]:
from synthesize import *
from preprocess import *
import random
random.seed(24)


config_folder = "configs/Chicago"
n = 100

person = process_pums_data(config_folder)
population_sample = synthesize_population(
    config_folder=config_folder, n_sample=n, min_age=16
)
ploc = puma_locations(config_folder)

output = []
attribute_descriptions = get_attribute_descriptions(person)
for i, individual in population_sample.reset_index().iterrows():
    individual_attributes = attribute_decoder_dict(individual.to_dict(), person)
    bio = write_individual_bio(individual_attributes, attribute_descriptions, config_folder, ploc=ploc)
    output.append(bio)
    
import langroid as lr
import langroid.language_models as lm

model_config, questions = load_config("configs/Chicago")
llm_config = lm.OpenAIGPTConfig(**model_config["model"])

sample_questions = ["AGE", "AAGE", "SEX", "LIC", "RACE", "NOGOWHY2", "JOBS", "WRKHRS", "OCCUP", "OCCUP_O", "INDUS"]
formatted_questions = []
for question in sample_questions:
    queued_question = questions[question]["question"]
    queued_responses = questions[question]["response"]
    
    items = list(queued_responses.items())
    random.shuffle(items)
    shuffled_responses = dict(items)
    
    if question == "DUMMY":
        formatted_question = f"{queued_question}"
    else:
        formatted_question = f"{queued_question} Choices: {shuffled_responses}"
    formatted_questions.append(formatted_question)

In [ ]:
formatted_questions
system_message = "You are a 43 year old woman of irish descent that lives in Chicago Ridge, Illinois. You were born in the U.S. in 1972 in the state of Illinois and live with a biological son or daughter. The date is July 17, 2015. Please answer the following travel survey questions to the best of your ability."
# system_message = "We are roleplaying "+output[70]+ "The date is July 17, 2015. Please answer the following travel survey questions to the best of your ability."

agent_config = lr.ChatAgentConfig(llm=llm_config, system_message=system_message)
agent = SurveyAgent(config=agent_config)
agent.enable_message(singleAnswerTool)
agent.enable_message(multipleAnswerTool)
agent.enable_message(discreteNumericTool)

agent.llm_response("how are you ?")
for question in formatted_questions:
    print(question)
    agent.llm_response(question)

In [ ]:
agent.message_history[6].content

In [ ]:
# with open("private/hf-key", "r") as f:
#     key = f.read().rstrip("\n")
# import tokenizers
# from tokenizers import Tokenizer

# tokenizer = Tokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", token=key)

import langroid as lr
import langroid.language_models as lm
from typing import Dict
from langroid.agent.chat_document import ChatDocument


def _select_choice(questions: Dict[str,str])->str:
    return list(questions.keys())[0]

llm_config = lm.OpenAIGPTConfig(
    chat_model="ollama/qwq",
    chat_context_length =20_000
)

agent_config = lr.ChatAgentConfig(
    llm=llm_config,
    system_message="You are a 25 year old athletic woman named Yulissa Gomez. Please answer the following questions.",
)



from typing import List, Optional
import langroid as lr
from langroid.agent.tool_message import ToolMessage
import langroid.language_models as lm

llm_config = lm.OpenAIGPTConfig(
    chat_model = "ollama/llama3.1",
    chat_context_length=16_00
)

config = lr.ChatAgentConfig(
    name="SurveyAgent",
    llm=llm_config,
    system_message="You are a 25 year old athletic female. Please answer the following survey questions related to fitness.",
    use_tools=True,
    use_functions_api=False,
    vecdb=None
)

class singleAnswerTool(lr.agent.ToolMessage):
    request: str = "singleChoiceResponse"
    purpose: str = """Please read the provided options and select ONE answer that best applies to you. Only specify the <answer_key>, and not the description."""
    answer_key: int

    @classmethod
    def example(cls):
        return [
            (
                'How many times per week do you get groceries? {\'-8\': "I don\'t know", \'-7\': \'I prefer not to answer\', \'-1\': \'Appropriate skip\', \'1\': \'1 day a week\', \'2\': \'2 days a week\', \'3\': \'3 days a week\', \'4\': \'4 days a week\', \'5\': \'5 days a week\', \'6\': \'6 days a week\', \'7\': \'7 days a week\'}',
                cls(answer_key=1)
            )
        ]


class SurveyAgent(lr.ChatAgent):
    def __init__(self, config: lr.ChatAgentConfig):
        super().__init__(config)
        self.tool_expected = False
        self.answersIndices = []
        self.answers = []
        self.queued_question: dict


    def singleChoiceResponse(self, msg: singleAnswerTool ) -> str:
        # return a single possible answer from a set of questions
        return str(msg.answer_index)


    def llm_response(
        self, message: Optional[str | ChatDocument] = None
    ) -> Optional[ChatDocument]:
        self.tool_expected = True
        return super().llm_response(message)

    def user_response(
        self,
        msg: Optional[str | ChatDocument] = None,
    ) -> Optional[ChatDocument]:
        self.tool_expected = False
        return super().user_response(msg)

agent = SurveyAgent(config=config)
agent.enable_message(singleAnswerTool)


In [ ]:
questions = {
    "-8": "I don't know",
    "-7": "I prefer not to answer",
    "-1": "Appropriate skip",
    "1": "1 day a week",
    "2": "2 days a week",
    "3": "3 days a week",
    "4": "4 days a week",
    "5": "5 days a week",
    "6": "6 days a week",
    "7": "7 days a week"
}

In [ ]:
response = agent.llm_response(message=f"How many days per week do you work out? Please select one the following: {questions}")

In [ ]:
agent.llm_response("Please tell us how you are feeling today?")

In [ ]:
race_questions = {
    "-8": "I don't know ",
    "-7": "I prefer not to answer ",
    "1": "White ",
    "2": "African American, Black",
    "3": "Asian ",
    "4": "American Indian, Alaskan Native",
    "5": "Native Hawaiian or Pacific Islander",
    "6": "Multiracial",
    "97": "Some other race "
    }

In [ ]:
agent.llm_response(f"Which option best describes you? {race_questions}")

In [ ]:
agent.message_history[-1].content